In [96]:
import ast
import re
from collections import defaultdict
from pathlib import Path

In [97]:
def replace_strings_in_module(module_ast, replacements):
    """Replace strings in a module with new values."""

    # Define a class to replace string nodes
    class StringReplacer(ast.NodeTransformer):
        def visit_Str(self, node):
            if node.s in replacements:
                node.s = replacements[node.s]
            return node

    # Traverse the AST and replace string nodes
    module_ast = StringReplacer().visit(module_ast)

    # Convert the AST back into code
    unparsed = ast.unparse(module_ast)

    # save the file adding "replace" to the name of the input file
    output_file = file_path.with_name(f"{p.stem}_replaced.py")
    with open(output_file, "w") as f:
        f.write(unparsed)

In [196]:
def extract_strings_from_module(module_path):
    """Extract all string literals from a module."""
    
    module_ast = ast.parse(module_path.read_text())

    def collect_strings(node):

        if isinstance(node, list):
            return [node]

        if isinstance(node, ast.Str) and not isinstance(node, ast.Expr):
            # If the node is a string literal, add its value to the list of strings
            return [{"KYE_TO_REPLACE":node.s}]
        
        elif isinstance(node, ast.JoinedStr):
            # If the node is a JoinedStr, join its values to get the whole string
            whole_string = ''
            for value in node.values:
                if isinstance(value, ast.Str):
                    whole_string += value.s
                elif isinstance(value, ast.FormattedValue):
                    value_strings = collect_strings(value.value)
                    if value_strings:
                        whole_string += '{' + value_strings[0] + '}'
                    else:
                        whole_string += '{}'
            return [whole_string]

        else:
            # Otherwise, recursively collect the string nodes from the node's children
            return sum((collect_strings(child) for child in ast.iter_child_nodes(node)), [])

    # Traverse the AST and collect all the string nodes
    strings = collect_strings(module_ast)

    return strings


In [197]:
def find_strings_in_folder(folder_path, output_path):
    """Find all string literals in all Python files in a folder."""
    
    string_dict = defaultdict(list)

    for py_file in Path(folder_path).rglob('*.py'):
        if any(
            [
                py_file.name.startswith('.'),
                '/.' in str(py_file),
                py_file.name.startswith('__'),
                "stackcomposed" in py_file.parts,
            ]
        ):
            continue
        
        # Find all the string literals in the file
        strings = extract_strings_from_module(py_file)

        # Add the strings to the dictionary with the file name as key
        for s in strings:
            string_dict[py_file.name].append(s)

    with open(output_path, 'w') as f:
        for key, value in string_dict.items():
            f.write(f"{key}:\n")
            for string in value:
                f.write(f"\t{string}\n")

In [198]:
p = Path("/home/dguerrero/1_modules/sepal_pysmm/component/widget/custom_drawer.py")
output_path = Path("/home/dguerrero/1_modules/sepal_pysmm/keys.txt")

In [199]:
find_strings_in_folder("/home/dguerrero/1_modules/sepal_pysmm/", output_path)

TypeError: can only concatenate str (not "dict") to str

In [132]:
# replacements = {
#     '\n        Display the apropriate tiles when the item is clicked.\n        \n        The tile to display will be all tile in the list with the mount_id as the current object.\n\n        Args:\n        ----\n            tiles ([sw.Tile]) : the list of all the available tiles in the app\n\n        Return:\n        ------\n            self\n        ': "name_x"
# }
# module_ast = ast.parse(p.read_text())

# replace_strings_in_module(p.read_text(), replacements)
